<a href="https://colab.research.google.com/github/sai-madala/Supervised-M_Learning-Models/blob/main/Log_Reg__Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**About the Dataset:**

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





**Importing the Dependencies**

In [ ]:
import numpy as np
import pandas as pd

import re             # re means Regular expression. It is very useful for searching words in a text/paragraph.
from nltk.corpus import stopwords   # Stopwords means those words(a, and, the, where, what, when) which doesn't add much value to a paragraph
from nltk.stem.porter import PorterStemmer    # Stemming is the process of reducing a word to its Root word. example: actor, actress, acting --> act
from sklearn.feature_extraction.text import TfidfVectorizer   # tfidf vector which is used to convert text into feature vectors/numbers.

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# import the nltk to download stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# print the stopwords in English

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**Data Pre-processing**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load the dataset to a pandas DataFrame

news_dataset = pd.read_csv('//content/drive/MyDrive/Colab Notebooks/Data sets/Fake news data_train.csv')

In [ ]:
news_dataset.shape

(20800, 5)

In [ ]:
# print the first 5 rows of the dataframe

news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# count the number of missing values in the dataset

news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# replacing the null values with empty string

news_dataset = news_dataset.fillna('')

In [ ]:
# merging the author name and news title

news_dataset['content'] = news_dataset['author']+' '+ news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


This content and label column dataset is used for prediction.

In [ ]:
# separating the data & label

X = news_dataset.drop(columns='label', axis=1)    # drop function is used to seperate the label column and remaining columns from a dataset
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

^ means exclusion. i.e it removes the variables other than a-z,A-Z. comma, space will be replaced with removed variables.
lower function replaces all the uppercase  variables
split function splits the variables.

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
#separating the data and label

X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [ ]:
X.shape

(20800,)

In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
Y.shape

(20800,)

In [ ]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer()    # Tf - term frequency, idf - inverse document frequency 
vectorizer.fit(X)

X = vectorizer.transform(X)

Inverse freq function finds those values which are repeating so many timesand it detects those words are not significant and reduces its importance value.

Term freq function finds the no.of repeating significant words and it will give a particular value to it.

by this they create a feature vectors/numbers. 

In [ ]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

**Splitting the dataset to training & test data**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

0.2 means 20% of test data is used and it is stored in the X test. 
The training data uses 80%.
The labels for Y train is stored in X train and the labels for Ytest is stored in X test.

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(20800, 5) (16640, 17128) (4160, 17128)


In [ ]:
print(Y.shape, Y_train.shape, Y_test.shape)

(20800,) (16640,) (4160,)


In [ ]:
print(X_train)
print(Y_train)

  (0, 16996)	0.09995656818816077
  (0, 16928)	0.3360072514082535
  (0, 15295)	0.09807665903342763
  (0, 13914)	0.3334797245354899
  (0, 13014)	0.2680313811122545
  (0, 12501)	0.3929876463935473
  (0, 11936)	0.24142639024498436
  (0, 10306)	0.09662001419895176
  (0, 10219)	0.3019527708144002
  (0, 3155)	0.3400831511004003
  (0, 2794)	0.3776836172783757
  (0, 336)	0.3360072514082535
  (1, 16996)	0.07263181421455335
  (1, 15424)	0.22579404836928033
  (1, 15417)	0.26613170238131584
  (1, 15295)	0.07126580880898774
  (1, 13453)	0.3387500815971264
  (1, 11421)	0.3084666283145136
  (1, 10306)	0.07020736153621741
  (1, 10061)	0.24924889065491332
  (1, 9856)	0.3387500815971264
  (1, 5313)	0.2768869285533855
  (1, 2761)	0.3387500815971264
  (1, 2207)	0.3387500815971264
  (1, 956)	0.2855580802628186
  :	:
  (16637, 10008)	0.32617532466383187
  (16637, 9588)	0.184164927958451
  (16637, 8153)	0.3214731053599403
  (16637, 5009)	0.3538476463585765
  (16637, 1868)	0.40919228974806576
  (16637, 364)	0.

**Training the Model: Logistic Regression**

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

**Evaluation**

**To find the accuracy score of Test and train data**

In [ ]:
# accuracy score on the training data

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9865985576923076


In [ ]:
# accuracy score on the test data

X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


accuracy of test data is more important than train data.

**Making a Predictive System to predict whether the news is Real or fake**

In [ ]:
X_new = X_test[3]     # 3 indicates the row number, to predict the row is fake or real

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [ ]:
print(Y_test[3])

0


Confusion Matrix

In [ ]:
Y_prediction = model.predict(X_test)
pd.crosstab(Y_test, Y_prediction, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,2004,73,2077
1,14,2069,2083
All,2018,2142,4160
